# download data:
i was unable to find ftp login info, so this has to be done by hand ( if you find a way around it lmk) 

i also could not get the noaa python package to work - https://pypi.org/project/noaa-weather/

the dataset we are using comes from here: https://www.ncei.noaa.gov/metadata/geoportal/rest/metadata/item/gov.noaa.ncdc:C00861/html

1. you can read on the features of the library and the important files here: ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt
2. the main directory you will be using is ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/
3. recommend that you download ghcnd-stations.txt, ghcnd-states.txt, and ghcnd_ghc.tar.gz  (previously ghcnd_all.tar.gz  but that is 10 times larger)
4. this notebook deals specifically with the ghcnd_ghc.tar.gz , you will need to have the contents un-zipped into a location on your pc


In [ ]:
#https://stackoverflow.com/questions/62165172/convert-dly-files-to-csv-using-python
import glob
import csv
import os

# fields as given by the spec

fields = [
    ["ID", 1, 11],
    ["YEAR", 12, 15],
    ["MONTH", 16, 17],
    ["ELEMENT", 18, 21]]

offset = 22

for value in range(1, 32):
    fields.append((f"VALUE{value}", offset,     offset + 4))
    fields.append((f"MFLAG{value}", offset + 5, offset + 5))
    fields.append((f"QFLAG{value}", offset + 6, offset + 6))
    fields.append((f"SFLAG{value}", offset + 7, offset + 7))
    offset += 8

# Modify fields to use Python numbering
fields = [[var, start - 1, end] for var, start, end in fields]
fieldnames = [var for var, start, end in fields]


# the goal of this code is to make 1 file TOTAL from many (1 per station), which can then be filtered  by year. 

# enter where you want a csv saved - it will be many Gigs
csv_filename = 'F:\weather\hcn\ghcnd_hcn.csv'

with open(csv_filename, 'w', newline='') as f_csv:
    
    # glob.glob should aim to where you unzipped the ghcnd_ghc.tar.gz , inside that will be ghcnd_hcn folder, which contains all the dailies
    for dly_filename in glob.glob(r'F:\weather\hcn\ghcnd_hcn\*.dly', recursive=True): 
        path, name = os.path.split(dly_filename)

        
        # you could replace this with adding to a dataframe or something else, but i am running out of brain power.
        with open(dly_filename, newline='') as f_dly:
            spamwriter  = csv.writer(f_csv)
            spamwriter.writerow(fieldnames) 

            for line in f_dly:
                row = [line[start:end].strip() for var, start, end in fields]
                spamwriter.writerow(row)
                
                
# end product is a csv with us weather station data.  DEFINTELY read the readme, this dataformat is bonkers.   ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt
